In [218]:
import geopandas as gpd 

In [219]:
tiles = gpd.read_file("../data/tiles.geojson")
trees_data = gpd.read_file("../data/trees_box.geojson")

In [220]:
tiles.to_crs(epsg=4326, inplace=True)
trees_data.to_crs(epsg=4326, inplace=True)

In [221]:
import re

def parse_tile_id(tile_id_str):
    match = re.match(r"Tile\(x=(\d+), y=(\d+), z=(\d+)\)", tile_id_str)
    if match:
        return match.groups()
    raise ValueError(f"Cannot parse tile ID: {tile_id_str}")

### converts it to geojson labels for each tiles

In [222]:
import os
from pathlib import Path

def split_geojson_by_tiles(trees_gdf, tiles_gdf, output_dir, prefix="OAM"):
    """Clip trees by tiles and save as individual GeoJSON files"""
    os.makedirs(output_dir, exist_ok=True)
    
    stats = {'processed': 0, 'skipped': 0, 'errors': 0, 'total_trees': 0}
    
    for idx, tile in tiles_gdf.iterrows():
        try:
            tile_geom = tile.geometry
            tile_id = tile['id']
            
            x, y, z = parse_tile_id(tile_id)
            tile_filename = f"{prefix}-{x}-{y}-{z}.geojson"
            
            intersecting_trees = trees_gdf[trees_gdf.intersects(tile_geom)].copy()
            
            if intersecting_trees.empty:
                stats['skipped'] += 1
                continue
            
            clipped_trees = gpd.clip(intersecting_trees, tile_geom)
            
            output_path = Path(output_dir) / tile_filename
            clipped_trees.to_file(output_path, driver="GeoJSON")
            
            stats['processed'] += 1
            stats['total_trees'] += len(clipped_trees)
            
        except Exception as e:
            print(f"Error processing tile {idx}: {e}")
            stats['errors'] += 1
            continue
    
    return stats

In [223]:
stats = split_geojson_by_tiles(trees_data, tiles, "../data/labels")

In [224]:
stats

{'processed': 458, 'skipped': 93, 'errors': 0, 'total_trees': 12337}

#### geojson Labels to YOLO Format

In [225]:
from glob import glob
import rasterio
import yaml

chips_dir = Path("../data/chips")
labels_dir = Path("../data/labels")
yolo_dir = Path("../data/yolo")
yolo_dir.mkdir(exist_ok=True)

yolo_labels_dir = Path(os.path.join(yolo_dir, "labels"))

yolo_labels_dir.mkdir(exist_ok=True)

label_files = sorted(labels_dir.glob("*.geojson"))


In [226]:
len(label_files)

458

In [227]:
classes = sorted(trees_data['species_mapped'].unique())
class_to_id = {cls: idx for idx, cls in enumerate(classes)}

print("Class mapping:")
for cls, idx in class_to_id.items():
    print(f"  {idx}: {cls}")

Class mapping:
  0: Banana
  1: Coconut
  2: Mango
  3: Papaya


In [228]:
def geojson_to_yolo(geojson_path, image_path, class_mapping):
    trees = gpd.read_file(geojson_path)
    
    with rasterio.open(image_path) as src:
        yolo_lines = []
        for _, tree in trees.iterrows():
            species = tree.get('species_mapped', 'Unknown')
            class_id = class_mapping.get(species, 0)
            
            minx, miny, maxx, maxy = tree.geometry.bounds
            
            top_py, top_px = src.index(minx, miny)
            bottom_py, bottom_px = src.index(maxx, maxy)
            
            center_x = (top_px + bottom_px) / 2 / src.width
            center_y = (top_py + bottom_py) / 2 / src.height
            width = abs(top_px - bottom_px) / src.width
            height = abs(top_py - bottom_py) / src.height
            
            yolo_lines.append(f"{class_id} {center_x:.6f} {center_y:.6f} {width:.6f} {height:.6f}")
    
    return yolo_lines

In [229]:
converted = 0
skipped = 0

for label_file in label_files:
    stem = label_file.stem
    image_file = chips_dir / f"{stem}.tif"
    
    if not image_file.exists():
        skipped += 1
        continue
    
    yolo_lines = geojson_to_yolo(label_file, image_file, class_to_id)
    
    yolo_file = yolo_labels_dir / f"{stem}.txt"
    with open(yolo_file, 'w') as f:
        f.write('\n'.join(yolo_lines))
    
    converted += 1

In [230]:
sample_label = next(labels_dir.glob("*.geojson"))
sample_tif = chips_dir / f"{sample_label.stem}.tif"

trees = gpd.read_file(sample_label)
with rasterio.open(sample_tif) as src:
    bounds = src.bounds
    print(f"Image bounds: {bounds}")
    print(f"Image shape: {src.shape}")
    
print(f"\nTree bounds sample:")
for _, tree in trees.head(3).iterrows():
    print(f"  {tree.geometry.bounds}")

Image bounds: BoundingBox(left=-175.33973693847656, bottom=-21.098594269392095, right=-175.33905029296875, top=-21.097953653592086)
Image shape: (256, 256)

Tree bounds sample:
  (-175.33955914945852, -21.098594269392095, -175.33950525054146, -21.098593057171307)
  (-175.33973693847656, -21.098590242833424, -175.33970615054147, -21.098539957162323)
  (-175.33929974945852, -21.098557442838974, -175.33924585054146, -21.098507157156764)


#### train val split 

In [231]:
import shutil
import pandas as pd
from PIL import Image

train_dir = yolo_dir / "train"
val_dir = yolo_dir / "val"
train_dir.mkdir(exist_ok=True)
val_dir.mkdir(exist_ok=True)

data = []
for label_file in labels_dir.glob("*.geojson"):
    trees = gpd.read_file(label_file)
    if not trees.empty:
        dominant_species = trees['species_mapped'].value_counts().idxmax()
        data.append({'file': label_file.stem, 'species': dominant_species})

df = pd.DataFrame(data)
train_df = df.groupby('species', group_keys=False).apply(lambda x: x.sample(frac=0.8, random_state=42))
val_df = df.drop(train_df.index)

for split_name, split_df, target_dir in [("train", train_df, train_dir), ("val", val_df, val_dir)]:
    for stem in split_df['file']:
        with rasterio.open(chips_dir / f"{stem}.tif") as src: Image.fromarray(src.read([1,2,3]).transpose(1,2,0)).save(target_dir / f"{stem}.png")
        shutil.copy(yolo_labels_dir / f"{stem}.txt", target_dir / f"{stem}.txt")

/tmp/ipykernel_411191/3085818379.py:18: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_df = df.groupby('species', group_keys=False).apply(lambda x: x.sample(frac=0.8, random_state=42))


In [232]:
print(f"Train: {len(train_df)} | Val: {len(val_df)}")

print(train_df['species'].value_counts().sort_index())

print(val_df['species'].value_counts().sort_index())

Train: 366 | Val: 92
species
Banana       2
Coconut    346
Mango       16
Papaya       2
Name: count, dtype: int64
species
Banana      1
Coconut    86
Mango       4
Papaya      1
Name: count, dtype: int64


In [233]:
data_config = {
    'path': str(yolo_dir.absolute()),
    'train': 'train',
    'val': 'val',
    'names': {idx: name for name, idx in class_to_id.items()}
}

config_file = yolo_dir / "config.yaml"
with open(config_file, 'w') as f:
    yaml.dump(data_config, f, sort_keys=False)
